<a href="https://colab.research.google.com/github/carlibeisel/Drains_Lower_Boise_River/blob/main/05_compile_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: July 21, 2022

Date modified: May 13, 2024

Purpose: Merges flow, climate, and land use annual stats together for drains.

In [ ]:
## --------------- ##
## IMPORT PACKAGES ##
## --------------- ##

import pandas as pd # to work with dataframe
import os # for file paths
import glob # read in a folder of csv
import numpy as np # basic statistics

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## ------------------------------ ##
## IMPORT ALL DATASETS            ##
## ------------------------------ ##

drains = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/drainage_flows_out/annual_flow_vals.csv')

land_files = sorted(glob.glob('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/landcover_calculations_out/final_metrics/*.csv'))
land = []
for i in land_files:
  land.append(pd.read_csv(i))
land = pd.concat(land)

clim_files = sorted(glob.glob('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/final/*.csv'))
clim = []
for i in clim_files:
  clim.append(pd.read_csv(i))
clim = pd.concat(clim)

hydromet = pd.read_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_output/hydromet_data_out/model_input_hydromet.csv')

ubrb_prcp = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')
ubrb_prcp = ubrb_prcp.rename(columns={'year': 'Year'}) #rename to fit column structure
ubrb_prcp = ubrb_prcp.rename(columns = {'precip':'ubrb_prcp'}) #rename to fit column structure

pivot_change = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/irrigation_change/irrigation_model_input.csv')
pivot_change = pivot_change.rename(columns = {'Predicted_Proportion': 'pivot_prop'}) #rename to fit column structure

water_rights = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/water_rights/final_wr.csv')
water_rights['Year'] = pd.to_numeric(water_rights['Year'], errors='coerce') # Filter rows where 'Year' is between 1987 and 2020
#water_rights = water_rights[(water_rights['Year'] >= 1987) & (water_rights['Year'] <= 2020)]
water_rights = water_rights.rename(columns = {'Drainshed': 'Name'}) #rename to fit column structure


In [ ]:
## ------------------------------------------ ##
## Use a dictionary to help align drain names ##
## ------------------------------------------ ##

relate = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drainage_flows/dain_relates.csv')
spatial = relate.dropna(subset=['Spatial Name'])
newnames = dict(zip(spatial['Spatial Name'], spatial['NewName']))

clim['NewName'] = clim['NAME'].map(newnames)
clim = clim.drop(['Unnamed: 0', 'NAME'], axis=1)
land['NewName2'] = land['DrainName'].map(newnames)
land = land.drop(['Unnamed: 0', 'DrainName'], axis=1)

In [ ]:
## ------------------------------ ##
## Merge the files together       ##
## ------------------------------ ##

merge_df = drains.merge(clim, left_on = ['Year', 'Name'], right_on = ['Year', 'NewName'])
merge_df = merge_df.merge(land, left_on = ['Year', 'Name'], right_on = ['dates', 'NewName2'])
merge_df = merge_df.drop(['Unnamed: 0', 'NewName', 'dates',  'NewName2'], axis=1)
merge_df = merge_df.merge(hydromet, on='Year', how='inner')
merge_df = merge_df.merge(ubrb_prcp, on='Year', how='inner')

merge_df.to_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/compile_data_out/model_input_0822.csv')

In [ ]:
## ------------------------------ ##
## Add in WR & irrigation change
## ------------------------------ ##

df = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/compile_data_out/model_input_0822.csv')

df = df.merge(pivot_change, on=['Name','Year'], how='left')

df = df.merge(water_rights, on=['Name','Year'], how='left')
df.fillna(0, inplace=True)

df.to_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/compile_data_out/model_input_0822.csv')